<a href="https://colab.research.google.com/github/stefisha/StefanVelickovic_Omega_DS_InvestmentRounds/blob/main/Investment_rounds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
from google.colab import drive

In [20]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import pandas as pd

In [22]:
file_path = '/content/drive/MyDrive/Data Science Task - VegaIT/python_task_data.csv'

In [23]:
df = pd.read_csv(file_path)

### Loading the dataset

In [24]:
df.head()

,permalink,company,numEmps,category,city,state,fundedDate,raisedAmt,raisedCurrency,round
0,lifelock,LifeLock,NaN,web,Tempe,AZ,1-May-07,6850000,USD,b
1,lifelock,LifeLock,NaN,web,Tempe,AZ,1-Oct-06,6000000,USD,a
2,lifelock,LifeLock,NaN,web,Tempe,AZ,1-Jan-08,25000000,USD,c
3,mycityfaces,MyCityFaces,7.0,web,Scottsdale,AZ,1-Jan-08,50000,USD,seed
4,flypaper,Flypaper,NaN,web,Phoenix,AZ,1-Feb-08,3000000,USD,a


In [25]:
print(df.columns)

Index(['permalink', 'company', 'numEmps', 'category', 'city', 'state',
       'fundedDate', 'raisedAmt', 'raisedCurrency', 'round'],
      dtype='object')


### Preprocessing

In [28]:
# Fill missing values for numEmps using the median
df['numEmps'] = df['numEmps'].fillna(df['numEmps'].median())

# Convert fundedDate to datetime format
if 'fundedDate' in df.columns:
    df['fundedDate'] = pd.to_datetime(df['fundedDate'], errors='coerce')

    # Create a new column for fundedYear from the fundedDate
    df['fundedYear'] = df['fundedDate'].dt.year

    # Drop rows with missing fundedDate values
    df = df.dropna(subset=['fundedDate'])

    # Drop unwanted columns
    df = df.drop(columns=['permalink', 'company', 'fundedDate'])

# Convert categorical columns into dummy variables
columns_to_dummy = ['category', 'city', 'state', 'raisedCurrency']
available_columns = [col for col in columns_to_dummy if col in df.columns]

# Apply get_dummies only to available columns
df = pd.get_dummies(df, columns=available_columns, drop_first=True)

We might consider dropping these features:
### 1. Dropping `permalink`:
- **Reason**: The `permalink` column typically contains a unique identifier for each company or record, similar to an ID. While this is useful for tracking records, it doesn't contribute to the predictive power of a machine learning model.
- **Why drop?**: Since it doesn't provide any meaningful information for prediction, keeping it can add unnecessary complexity.

### 2. Dropping `company`:
- **Reason**: The `company` column usually contains the name of the company. The company name is categorical data, but it likely has no direct influence on the prediction target (such as funding round type).
- **Why drop?**: Similar to `permalink`, the company name doesn't add value to the model. Using it could overfit the model to specific company names, which is not desirable in most cases.

### 3. Dropping `fundedDate`:
- **Reason**: In the preprocessing step, you extract the year (`fundedYear`) from the `fundedDate`, which is likely more relevant for prediction.
- **Why drop?**: After extracting the useful part (the year), the full date isn't needed for the prediction task, and keeping it might add unnecessary complexity. The exact timestamp of the funding event typically does not provide additional predictive power compared to just the year.

### Summary:
- **`permalink` and `company`**: These columns act as identifiers and are not helpful for predicting the target variable.
- **`fundedDate`**: We already extracted the important information (the year), so the full date is no longer needed.

In [31]:
print(df.columns)

Index(['numEmps', 'raisedAmt', 'round', 'fundedYear', 'category_cleantech',
       'category_consulting', 'category_hardware', 'category_mobile',
       'category_other', 'category_software',
       ...
       'state_OR', 'state_PA', 'state_RI', 'state_TN', 'state_TX', 'state_UT',
       'state_VA', 'state_WA', 'raisedCurrency_EUR', 'raisedCurrency_USD'],
      dtype='object', length=237)


In [32]:
df.head()

,numEmps,raisedAmt,round,fundedYear,category_cleantech,category_consulting,category_hardware,category_mobile,category_other,category_software,...,state_OR,state_PA,state_RI,state_TN,state_TX,state_UT,state_VA,state_WA,raisedCurrency_EUR,raisedCurrency_USD
0,20.0,6850000,b,2007,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,20.0,6000000,a,2006,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,20.0,25000000,c,2008,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3,7.0,50000,seed,2008,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,20.0,3000000,a,2008,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


### Modeling

In [33]:
# Defining the features (X) and target variable (y)
X = df.drop(columns=['round'])
y = df['round']

In [40]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder

In [35]:
# Encode the target variable (round)
le = LabelEncoder()
y = le.fit_transform(y)

In [36]:
# Split the data into training and test sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [37]:
# Train a basic Random Forest model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [38]:
# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=le.classes_)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [39]:
# Output the results
print(f"Model Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:")
print(report)

Model Accuracy: 45.21%
Classification Report:
              precision    recall  f1-score   support

           a       0.53      0.61      0.57       122
       angel       0.41      0.33      0.37        27
           b       0.32      0.42      0.36        74
           c       0.43      0.22      0.29        27
           d       0.33      0.10      0.15        10
  debt_round       0.00      0.00      0.00         4
           e       0.00      0.00      0.00         1
        seed       0.67      0.42      0.51        24
unattributed       0.00      0.00      0.00         3

    accuracy                           0.45       292
   macro avg       0.30      0.23      0.25       292
weighted avg       0.45      0.45      0.44       292



In [46]:
def calculate_and_print_metrics(y_true, y_pred):
    """
    Calculate and print evaluation metrics: accuracy, precision, recall, and F1-score.

    Parameters:
    - y_true: The true labels
    - y_pred: The predicted labels
    """
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, average='weighted'),
        'recall': recall_score(y_true, y_pred, average='weighted'),
        'f1_score': f1_score(y_true, y_pred, average='weighted')
    }

    # Print the metrics
    print(f"Accuracy: {metrics['accuracy']:.2f}")
    print(f"Precision: {metrics['precision']:.2f}")
    print(f"Recall: {metrics['recall']:.2f}")
    print(f"F1 Score: {metrics['f1_score']:.2f}")

In [47]:
from xgboost import XGBClassifier
# Step 6: Train the XGBoost model
model = XGBClassifier(random_state=42)
model.fit(X_train, y_train)

# Step 7: Make predictions and evaluate the model
y_pred = model.predict(X_test)

In [48]:
# Calculate and print the metrics
calculate_and_print_metrics(y_test, y_pred)

# Print detailed classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

Accuracy: 0.49
Precision: 0.46
Recall: 0.49
F1 Score: 0.47

Classification Report:
              precision    recall  f1-score   support

           a       0.59      0.69      0.63       122
       angel       0.50      0.30      0.37        27
           b       0.39      0.46      0.42        74
           c       0.24      0.19      0.21        27
           d       0.00      0.00      0.00        10
  debt_round       0.00      0.00      0.00         4
           e       0.00      0.00      0.00         1
        seed       0.61      0.46      0.52        24
unattributed       0.00      0.00      0.00         3

    accuracy                           0.49       292
   macro avg       0.26      0.23      0.24       292
weighted avg       0.46      0.49      0.47       292



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

### 1. Handle Class Imbalance
- **Problem**: The dataset might have imbalanced classes where some funding rounds (e.g., 'angel', 'seed') have fewer samples compared to others (e.g., 'a', 'b').
- **Solution**: Apply **SMOTE (Synthetic Minority Over-sampling Technique)** to generate synthetic samples for underrepresented classes and balance the dataset.

### 2. Hyperparameter Tuning
- **Problem**: The default hyperparameters may not be optimal for the XGBoost model, limiting its performance.
- **Solution**: Use **RandomizedSearchCV** or **GridSearchCV** to tune hyperparameters and find the best set of parameters for the model, improving accuracy, precision, recall, and F1-score.

### 3. Cross-Validation
- **Problem**: The performance on a single test set might not generalize well to unseen data.
- **Solution**: Use **cross-validation** (such as 5-fold or 10-fold) to evaluate the model on multiple subsets of the data and get a more robust performance estimate.

### 4. Feature Scaling
- **Problem**: Features with large variations in value (like `raisedAmt`) can negatively impact the model's performance.
- **Solution**: Normalize the feature values using **MinMaxScaler** or **StandardScaler**, especially for algorithms sensitive to feature magnitude (like XGBoost).

### 5. Feature Importance and Selection
- **Problem**: Some features might not contribute much to the prediction, while others might have more predictive power.
- **Solution**: Analyze **feature importance** after training and consider selecting the most important features or removing less important ones to improve model efficiency.